In [1]:
#import packages
import pandas as pd
import numpy as np
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from keras.models import Sequential

# Date time conversion registration
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#to plot within notebook
import matplotlib.pyplot as plt
%matplotlib inline

#setting figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 35,20

#for normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

df = pd.read_csv('AAPL.csv')
df.index = df['Date']
#importing required libraries
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM


df.head()

Using TensorFlow backend.


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-01-22,2015-01-22,110.260002,112.470001,109.720001,112.400002,103.275063,53796400
2015-01-23,2015-01-23,112.300003,113.750000,111.529999,112.980003,103.807991,46464800
2015-01-26,2015-01-26,113.739998,114.360001,112.800003,113.099998,103.918228,55615000
2015-01-27,2015-01-27,112.419998,112.480003,109.029999,109.139999,100.279716,95568700
2015-01-28,2015-01-28,117.629997,118.120003,115.309998,115.309998,105.948822,146477100


In [2]:
#creating dataframe
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

In [3]:
#setting index
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

In [4]:
#creating train and test sets
dataset = new_data.values
print(dataset)

[[112.400002]
 [112.980003]
 [113.099998]
 ...
 [315.23999]
 [318.730011]
 [316.570007]]


In [11]:
train = dataset[0:987,:]
valid = dataset[987:,:]
print(valid)

[[150.729996]
 [146.830002]
 [157.169998]
 [156.149994]
 [156.229996]
 [157.740005]
 [157.919998]
 [142.190002]
 [148.259995]
 [147.929993]
 [150.75]
 [153.309998]
 [153.800003]
 [152.28999299999998]
 [150.0]
 [153.070007]
 [154.940002]
 [155.860001]
 [156.820007]
 [153.300003]
 [153.919998]
 [152.699997]
 [157.759995]
 [156.300003]
 [154.679993]
 [165.25]
 [166.440002]
 [166.520004]
 [171.25]
 [174.179993]
 [174.240005]
 [170.940002]
 [170.41000400000001]
 [169.429993]
 [170.889999]
 [170.179993]
 [170.800003]
 [170.419998]
 [170.929993]
 [172.029999]
 [171.059998]
 [172.970001]
 [174.229996]
 [174.330002]
 [174.869995]
 [173.149994]
 [174.970001]
 [175.850006]
 [175.529999]
 [174.520004]
 [172.5]
 [172.91000400000001]
 [178.899994]
 [180.91000400000001]
 [181.71000700000002]
 [183.729996]
 [186.119995]
 [188.020004]
 [186.529999]
 [188.16000400000001]
 [195.08999599999999]
 [191.050003]
 [188.740005]
 [186.78999299999998]
 [188.470001]
 [188.720001]
 [189.949997]
 [191.240005]
 [194.

In [12]:
#converting dataset into x_train and y_train
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))



In [14]:
print(x_train.shape)

(927, 60, 1)


In [ ]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2)

In [ ]:
#predicting 246 values, using past 60 from the train data

inputs = new_data[len(new_data) - len(valid) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = scaler.transform(inputs)

X_test = []
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
closing_price = model.predict(X_test)
closing_price = scaler.inverse_transform(closing_price)

In [ ]:
print(closing_price)
#X_test
#print(valid)

In [ ]:
rms=np.sqrt(np.mean(np.power((valid-closing_price),2)))
rms

In [ ]:
#setting index as date values
df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
df.index = df['Date']

#creating a separate dataset
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])

for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]
    

#create features
from fastai.tabular import  add_datepart
add_datepart(new_data, 'Date')
new_data.drop('Elapsed', axis=1, inplace=True)  #elapsed will be the time stamp


In [ ]:
#for plotting
    
train = new_data[:987]
valid = new_data[987:]
valid['Predictions'] = closing_price
plt.figure(figsize=(16,8))
plt.plot(train['Close'], color='green')
plt.plot(valid[['Close','Predictions']])

In [ ]:
valid